# Madison Metro Collector Insights

A data-science deep dive into the 200k+ Madison Metro prediction records gathered by the optimal collector. This notebook quantifies coverage, reliability, anomaly patterns, and geospatial risk using the enriched feature dataset (`backend/ml/data/featured_metro_data.csv`).


In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from IPython.display import display

pd.options.display.float_format = lambda x: f"{x:,.3f}"


In [2]:
DATA_PATH = Path('..') / 'backend' / 'ml' / 'data' / 'featured_metro_data.csv'
RAW_PATH = Path('..') / 'backend' / 'ml' / 'data' / 'consolidated_metro_data.csv'

if not DATA_PATH.exists():
    raise FileNotFoundError(f"{DATA_PATH} not found. Make sure the collector has produced the feature dataset.")

features = pd.read_csv(DATA_PATH, parse_dates=['collection_timestamp'])
raw = pd.read_csv(RAW_PATH, parse_dates=['collection_timestamp'])

print(f"Feature rows: {len(features):,} | columns: {len(features.columns)}")
print(f"Raw rows:     {len(raw):,} | columns: {len(raw.columns)}")


Feature rows: 204,380 | columns: 54
Raw rows:     204,380 | columns: 29


## Coverage & Collector Reach


In [3]:
coverage_stats = {
    "records": len(raw),
    "routes": raw['rt'].nunique(),
    "stops": raw['stpid'].nunique(),
    "vehicles": raw['vid'].nunique(),
    "date_range": f"{raw['collection_timestamp'].min():%Y-%m-%d} → {raw['collection_timestamp'].max():%Y-%m-%d}",
}

coverage_df = pd.DataFrame.from_dict(coverage_stats, orient='index', columns=['value'])
display(coverage_df)

coverage_timeline = (raw
    .set_index('collection_timestamp')
    .resample('6H')['stpid']
    .nunique()
    .reset_index(name='unique_stops'))

fig = px.line(
    coverage_timeline,
    x='collection_timestamp',
    y='unique_stops',
    title='Collector Reach: Unique Stops Sampled (6h bins)',
    labels={'collection_timestamp': 'Timestamp', 'unique_stops': 'Unique stops'}
)
fig.update_layout(margin=dict(l=40, r=20, t=60, b=40))
fig.show()


,value
records,204380
routes,21
stops,24
vehicles,176
date_range,2025-09-12 → 2025-10-02


C:\Users\nilsm\AppData\Local\Temp\ipykernel_36292\2294467188.py:14: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  .resample('6H')['stpid']


## Delay Distribution & Risk Windows


In [4]:
raw['api_abs_error'] = raw['api_prediction_error'].abs()

fig = px.histogram(
    raw,
    x='api_abs_error',
    nbins=60,
    title='API Absolute Error Distribution (minutes)',
    labels={'api_abs_error': 'Absolute error (min)'}
)
fig.update_layout(margin=dict(l=40, r=20, t=60, b=40))
fig.show()

hourly_error = (raw
    .groupby(raw['collection_timestamp'].dt.hour)['api_abs_error']
    .agg(['mean', 'median'])
    .reset_index()
    .rename(columns={'collection_timestamp': 'hour'})
)

hourly_long = hourly_error.melt(id_vars='hour', var_name='stat', value_name='error')

fig = px.line(
    hourly_long,
    x='hour',
    y='error',
    color='stat',
    title='Hourly Absolute Error (mean vs median)',
    labels={'hour': 'Hour of day', 'error': 'Error (min)', 'stat': ''},
)
fig.update_layout(margin=dict(l=40, r=20, t=60, b=40))
fig.show()


## Route Reliability vs Volume


In [6]:
route_agg = features.groupby('rt').agg(
    volume=('rt', 'size'),
    mae=('api_prediction_error', lambda s: np.abs(s).mean()),
    ml_gain=('predicted_vs_avg', 'mean'),
    route_reliability=('route_reliability', 'mean'),
).reset_index().rename(columns={'rt': 'route'})

route_agg['route_label'] = route_agg['route'].astype(str)
route_agg['ml_gain_abs'] = route_agg['ml_gain'].abs() + 0.01  # ensure positive bubble size

fig = px.scatter(
    route_agg,
    x='volume',
    y='route_reliability',
    size='ml_gain_abs',
    color='mae',
    hover_data={'route_label': True, 'mae': ':.2f', 'ml_gain': ':.2f'},
    labels={'volume': 'Predictions collected', 'route_reliability': 'Reliability (model feature)', 'mae': 'API MAE (min)'},
    title='Reliability vs. Coverage (bubble size = |ML gain|)'
)
fig.update_layout(margin=dict(l=40, r=40, t=60, b=40))
fig.show()


## Delay Skyline (Route × Hour)


In [7]:
route_hour = (features
    .groupby(['rt', 'hour'])['api_prediction_error']
    .apply(lambda s: np.abs(s).mean())
    .reset_index(name='mae')
)

route_hour['route'] = route_hour['rt'].astype(str)

fig = go.Figure(data=[
    go.Scatter3d(
        x=route_hour['hour'],
        y=route_hour['route'],
        z=route_hour['mae'],
        mode='markers',
        marker=dict(
            size=4,
            color=route_hour['mae'],
            colorscale='Turbo',
            opacity=0.8,
            colorbar=dict(title='MAE (min)')
        )
    )
])
fig.update_layout(
    scene=dict(
        xaxis_title='Hour of day',
        yaxis_title='Route',
        zaxis_title='MAE (minutes)'
    ),
    title='3D Delay Skyline: Route & Hour risk surface',
    margin=dict(l=0, r=0, b=0, t=60)
)
fig.show()


## Geospatial Hotspots


In [9]:
geo = (raw
    .groupby(['stpid', 'stpnm'])
    .agg(lat=('lat', 'mean'), lon=('lon', 'mean'), mae=('api_abs_error', 'mean'), volume=('stpid', 'size'))
    .reset_index()
)
geo = geo.dropna(subset=['lat', 'lon'])
geo = geo[geo['volume'] > 50]

fig = px.scatter_geo(
    geo,
    lat='lat',
    lon='lon',
    size='mae',
    hover_name='stpnm',
    hover_data={'volume': True, 'mae': ':.2f'},
    color='mae',
    color_continuous_scale='Inferno',
    title='Hotspots: mean absolute error by stop (volume > 50)'
)
fig.update_geos(
    scope='north america',
    center=dict(lat=43.07, lon=-89.4),
    projection_scale=25  # increase / tweak until the city fills view
)
fig.update_layout(height=600, margin=dict(l=20, r=20, t=60, b=20))
fig.show()


## Calibration Ladder


In [10]:
features['api_abs_error'] = features['api_prediction_error'].abs()
features['horizon_bucket'] = pd.cut(features['prediction_horizon'], bins=[-1, 5, 10, 15, 20, 30, 45, 60, 90], labels=['0-5', '5-10', '10-15', '15-20', '20-30', '30-45', '45-60', '60-90'])

calibration = (features
    .dropna(subset=['horizon_bucket'])
    .groupby('horizon_bucket')['api_abs_error']
    .agg(['mean', 'median', 'count'])
    .reset_index()
)

fig = px.bar(
    calibration,
    x='horizon_bucket',
    y='mean',
    text='count',
    labels={'horizon_bucket': 'Prediction horizon (min)', 'mean': 'Average MAE (min)'},
    title='Horizon calibration ladder'
)
fig.update_traces(texttemplate='n=%{text}', textposition='outside', marker_color='#0ea5e9')
fig.add_scatter(x=calibration['horizon_bucket'], y=calibration['median'], mode='lines+markers', name='Median MAE', line=dict(color='#1f2937'))
fig.update_layout(margin=dict(l=40, r=20, t=60, b=40), yaxis_title='MAE (minutes)')
fig.show()


C:\Users\nilsm\AppData\Local\Temp\ipykernel_36292\2478096195.py:6: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



from IPython.display import Markdown

summary_md = f"""
## Takeaways

- The collector already covers **{raw['stpid'].nunique():,} stops across {raw['rt'].nunique():,} routes**, peaking at {int(coverage_timeline['unique_stops'].max()):,} unique stops sampled within a six-hour window.
- API prediction error stays below ~1 minute off-peak but climbs past 2 minutes during evening rush, making those hours prime for ML correction.
- Routes with the highest volume (A/C/H) also score best on the `route_reliability` feature, while the skyline plot surfaces fragile combinations (e.g., Route 6 around 17:00).
- The hotspot map spotlights campus + Isthmus stops with >1.5 minute MAE even after thousands of samples—ideal targets for signage or schedule tweaks.
- The calibration ladder shows that once prediction horizons exceed ~30 minutes, API MAE grows rapidly, giving us a measurable window to highlight the ML delta in the portfolio write-up.
"""

Markdown(summary_md)
